In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5cf41d98269b1ef4fc91e8eb75f95a4a58e15a4aae5c149eed9c2dfb5008f650
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [40]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

import torch
from sentence_transformers import SentenceTransformer, models, util

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv
/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv
/kaggle/input/youtube-videos-title-description-comments/UScomments.csv
/kaggle/input/youtube-videos-title-description-comments/GB_category_id.json
/kaggle/input/youtube-videos-title-description-comments/US_category_id.json
/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv
/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv
/kaggle/input/youtube-videos-title-description-comments/USvideos.csv


In [3]:
device = 'cuda'
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
gb_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv', on_bad_lines='skip')
gb_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [5]:
gb_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv')
gb_text_df = gb_text_df.drop('Unnamed: 0', axis=1)
gb_text_df.head()

,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [6]:
gb_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv', on_bad_lines='skip')
gb_comments_df.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [7]:
us_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/USvideos.csv', on_bad_lines='skip')
us_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [8]:
us_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv')
us_text_df = us_text_df.drop('Unnamed: 0', axis=1)
us_text_df.head()

,id,title,description
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Grab your 365 DAY MAVERICK MERCH! Exclusively ...
1,cLdxuaxaQwc,My Response,NaN
2,WYYvHb03Eog,Apple iPhone X first look,The Apple iPhone X is the new super flagship p...
3,sjlHnJvXdQs,iPhone X (parody),MORE APPLE STUFF ► https://www.youtube.com/wat...
4,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,SUBSCRIBE: http://bit.ly/A24subscribe\n\nBased...


In [9]:
us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')
us_comments_df.head()

/tmp/ipykernel_31/3472227992.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [10]:
text_df = pd.concat([gb_text_df, us_text_df], axis=0)
print("Len of GB data - ", len(gb_text_df))
print("Len of US data - ", len(us_text_df))
print("Len of combined data - ", len(text_df))
text_df.head()

Len of GB data -  7120
Len of US data -  7227
Len of combined data -  14347


,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [11]:
comments_df = pd.concat([gb_comments_df, us_comments_df], axis=0)
print("Len of GB data - ", len(gb_comments_df))
print("Len of US data - ", len(us_comments_df))
print("Len of combined data - ", len(comments_df))
comments_df.head()

Len of GB data -  718452
Len of US data -  691400
Len of combined data -  1409852


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [13]:
len(comments_df.comment_text.unique())

622630

In [14]:
details_df = pd.concat([gb_detail_df, us_detail_df], axis=0)
print("Len of GB data - ", len(gb_detail_df))
print("Len of US data - ", len(us_detail_df))
print("Len of combined data - ", len(details_df))
details_df.head()

Len of GB data -  7993
Len of US data -  7992
Len of combined data -  15985


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


## Fine-Tuning

### Transformer(-based) and Sequential Denoising Auto-Encoder (TSDAE)

TSDAE is a pre-training method where noise is introduced to input sequences by deleting or swapping tokens/words. These noisy sentences are encoded by the transformer model into sentence vectors. Another decoder network then attempts to reconstruct the original input from the noisy sentence encoding.

Model -> 
**Noisy Sentence -> Enocder -> Sentence Embedding -> Decoder -> Predicted correct sentence.**

Masked Language Modelling (MLM) uses masking of random tokens. TSDAE is simiillar to that except, in TSDAE the decoder only has access to the sentence vector produced by the encoder, whereas in MLM decoder has access to full-length word embeddings for every single token.

As mentioned in the TSDAE paper, [CLS] token pooling can be used as the sentence embedding instead of a mean pooling.

### Preparing the dataset

In [21]:
splitter = re.compile(r'\.\s?\n?')
comments_text = '. '.join(comments_df.comment_text.astype(str).unique())
comments_text = splitter.split(comments_text)
comments_text = list(filter(lambda x: x.strip() != '', comments_text))
comments_text[:10]

["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10)",
 'To be there with a samsung phone\\n😂😂😂',
 'Thank gosh, a place I can watch it without having to be at HD',
 'my speed doesn’t support HD',
 'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
 'Power is the disease',
 ' Care is the cure',
 ' Keep caring for yourself and others as best as you can',
 ' This is life',
 'Keep calm and buy iphone 8 Keep calm and buy iphone 8 plus Keep calm and buy iphone X']

In [22]:
len(comments_text)

957177

Need to add noise to the sentences and create a pytorch Dataset object. sentence-transformers library provides DenoisingAutoEncoderDataset which automatically handles this.

[DenoisingAutoEncoderDataset](https://www.sbert.net/docs/package_reference/datasets.html#denoisingautoencoderdataset) automatically deletes tokens with a probability of 60% per token.

In [24]:
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader

sentences = comments_text[:100000]

train_data = DenoisingAutoEncoderDataset(sentences)
dataloader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)

In [25]:
bert = models.Transformer('bert-base-uncased')
pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')

model = SentenceTransformer(modules=[bert, pooling])
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

Using [DenoisingAutoEncoderLoss](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/DenoisingAutoEncoderLoss.py) loss class for loss function. It is basically cross entropy loss applied to a batch of noisy sentences and their corresponding to correct sentences.

In [26]:
from sentence_transformers.losses import DenoisingAutoEncoderLoss

loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.3.crossattention.output.dense.bias', 'bert.encoder.layer.6.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.2.crossattention.self.query.bias', 'bert.encoder.layer

In [27]:
model.fit(
    train_objectives=[(dataloader, loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('output/youtube-comments-tsdae-bert-base-uncased')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12500 [00:00<?, ?it/s]

### Testing the model

Cleaning up the description to remove any urls, etc.

In [28]:
from nltk.tokenize import RegexpTokenizer
import re
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

We are joining video title, channel title and description of the video to judge relevancy. As much of the text is there which relates to the video content, it'll be better to judge relevancy using the sentence embeddings.

In [29]:
def get_embeddings(model, data):
    return model.encode(data, convert_to_tensor=True)

Getting all the comments for a video.

In [55]:
comments = comments_df[comments_df['video_id'] == 'jt2OHQh0HoQ'].comment_text.unique()
comments[:10]

array(["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
       'To be there with a samsung phone\\n😂😂😂',
       'Thank gosh, a place I can watch it without having to be at HD... my speed doesn’t support HD',
       'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
       'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.',
       'Keep calm and buy iphone 8 Keep calm and buy iphone 8 plus Keep calm and buy iphone X. What is your favourite?',
       'i am a big fan of youtube and u !!!!!!!!!!!!!',
       'You will never find Losers who line up and pay good money for this crap line up to do charity or community work.',
       "*APPLE JUST COMMENTED ON MY LAST VIDEO* I'm crying right now 😭😭😭😢😢😢",
       "I'm only here to see Emma, I love her so much! i'm so proud of her! :)"],
      dtype=object)

In [56]:
len(comments)

398

In [57]:
comments_embeddings = get_embeddings(model, comments)
comments_embeddings.shape

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

torch.Size([398, 768])

In [58]:
cleaned_description = preprocess(text_df[text_df['id'] == 'jt2OHQh0HoQ'].iloc[0].description)
title = text_df[text_df['id'] == 'jt2OHQh0HoQ'].iloc[0].title
channel_title = details_df[details_df['video_id'] == 'jt2OHQh0HoQ'].channel_title.values[0]
query_text = title + '. ' + cleaned_description + '. ' + channel_title
query_text

'Live Apple Event - Apple September Event 2017 - iPhone 8, iPhone X, iOS 11 - Apple Keynote. live apple event apple september event iphone iphone ios apple keynote apple special event apple new iphone officially unveiled today latest smartphone expected radical change company seek wow users anniversary device. Apple Event'

In [59]:
query_embedding = get_embeddings(model, [query_text])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
hits = util.semantic_search(query_embedding, comments_embeddings, top_k=398)
hits = hits[0]

In [61]:
hits[:10]

[{'corpus_id': 302, 'score': 0.6865788698196411},
 {'corpus_id': 13, 'score': 0.6865194439888},
 {'corpus_id': 168, 'score': 0.6590517163276672},
 {'corpus_id': 231, 'score': 0.6545993089675903},
 {'corpus_id': 319, 'score': 0.6237219572067261},
 {'corpus_id': 304, 'score': 0.620091438293457},
 {'corpus_id': 119, 'score': 0.6190329194068909},
 {'corpus_id': 341, 'score': 0.617639422416687},
 {'corpus_id': 307, 'score': 0.6058382391929626},
 {'corpus_id': 228, 'score': 0.605422854423523}]

In [62]:
def print_top_10(hits):
    for hit in hits[:10]:
        print("Comment - ", comments[hit['corpus_id']])

In [63]:
def print_bottom_10(hits):
    for hit in hits[-10:]:
        print("Comment - ", comments[hit['corpus_id']])

In [64]:
print_top_10(hits)

Comment -  Showcase Offer: Free iPhone 7 GIVEAWAY 2017. Pick up a new apple iphone every year. To successfully register and get a FREE iphone move to google and type: *givefonesaway*
Comment -  3 years late on wireless charging, 3 years late on oled technology , old facial recognition tech enhanced by old IR tech. So your late on just about every front....what to do? I know, lets lose what makes our product instantly recognizable! This is Apples windows 8, Samsung are bound to be loving this.
Comment -  APPLE parody launch event -----   https://youtu.be/oMMPw-2yZco
Comment -  Hahaha stop making a fool of people guys with your gimmicks. Your face ID failed In front of trillion of viewers worldwide. And nothing that you showed on this event is new. Everything is a copy or even 10 year old tech. From other phones. The cut screen is from the phone called essential phone from Canada. What is it that you have developed. Anything at all? You need something new. You need me. Connect with me. A

In [65]:
print_bottom_10(hits)

Comment -  Goood  jod
Comment -  W h e r e  i s  t h e  \nh e a d p h o n e\nj a c k ?
Comment -  هل من عرب هنا☺
Comment -  Aye Aye
Comment -  اي عربي لايك
Comment -  fuck you beech
Comment -  Sup
Comment -  22:40:39
Comment -  بتييييييييييييزك
Comment -  birby


In [66]:
for hit in hits[100:120]:
        print("Comment - ", comments[hit['corpus_id']])

Comment -  I'm watching this with my iPhone 4s who is still alive in 3 years that's why I love iPhones they're good and longlasting😘❤️💕
Comment -  i love IPhone 8！！！！
Comment -  Put into the iPhone T-mode for deaf people so that sound will be heard through the hearing aid
Comment -  There could be a nice little advantage to the face recognition. You could unlock your phone with gloves on.😃 but you still need to swipe. 😳😬🤔
Comment -  *#MamaDijo1VideoAntesDeLaCama*
Comment -  Power is the disease.  Care is the cure.  Keep caring for yourself and others as best as you can.  This is life.
Comment -  अब RS 15000-50000/ महीना कमायें.  घर बैठे बिना एक भी पैसा लगाय आप जॉब कर सकते है ! न ही कुछ बेचना है न ही खरीदना ! बस Facebook aur WhatsApp अच्छे से चलना आता हो।। और आप के पास एक Android phone हो । तो लिखे JOB और भेज दे  (7078723010 whats app)
Comment -  an apple a day keep the doctor away
Comment -  https://youtu.be/OPwJ27XoGqc
Comment -  ANDROID IS THE BEST 👍
Comment -  https://youtu.be/9TZhV

We see a pattern that sentences with similar length to the query are getting scored highly.

Trying with pre-trained sentence transformer model.

In [67]:
model_2 = SentenceTransformer('bert-base-uncased', device=device)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [68]:
comments_embeddings_2 = get_embeddings(model_2, comments)
query_embedding_2 = get_embeddings(model_2, [query_text])

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
hits_2 = util.semantic_search(query_embedding_2, comments_embeddings_2, top_k=398)
hits_2 = hits_2[0]

In [70]:
print_top_10(hits_2)

Comment -  The iPhone x design is a rip off of  the Samsung Galaxy S4 and somehow apple thinks they're innovative by adding a facial Id but Samsung Galaxy S4 already did it apple is over expensive over rated shit
Comment -  Haha basically copying the Samsung s8 . Samsung had all of these features already.  Much better and much cheaper phone . Now outselling apple. Plus yours not locked into I tunes etc  . With samsung you download whatever app you want
Comment -  Showcase Offer: Free iPhone 7 GIVEAWAY 2017. Pick up a new apple iphone every year. To successfully register and get a FREE iphone move to google and type: *givefonesaway*
Comment -  So apple jas finally features that Samsung has for years ? Wow
Comment -  Man iPhone 8 is the worst there is no home button I will not buy apple 8
Comment -  3 years late on wireless charging, 3 years late on oled technology , old facial recognition tech enhanced by old IR tech. So your late on just about every front....what to do? I know, lets lo

In [71]:
print_bottom_10(hits)

Comment -  Goood  jod
Comment -  W h e r e  i s  t h e  \nh e a d p h o n e\nj a c k ?
Comment -  هل من عرب هنا☺
Comment -  Aye Aye
Comment -  اي عربي لايك
Comment -  fuck you beech
Comment -  Sup
Comment -  22:40:39
Comment -  بتييييييييييييزك
Comment -  birby


Similar pattern with bert-base-uncased without fine tuning.

In [73]:
model_3 = SentenceTransformer('all-MiniLM-L6-v2', device=device)
comments_embeddings_3 = get_embeddings(model_3, comments)
query_embedding_3 = get_embeddings(model_3, [query_text])

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
hits_3 = util.semantic_search(query_embedding_3, comments_embeddings_3, top_k=398)
hits_3 = hits_3[0]

In [75]:
print_top_10(hits_3)

Comment -  The best thing to ever happen to iPhone
Comment -  I can't wait for the iPhone x
Comment -  iPhone X or 10 looks amazing, looking forward to get that. But I'm not really amazed by the iPhone 8 even though I have an iPhone 6+
Comment -  Why no iPhone 9
Comment -  The iPhone x design is a rip off of  the Samsung Galaxy S4 and somehow apple thinks they're innovative by adding a facial Id but Samsung Galaxy S4 already did it apple is over expensive over rated shit
Comment -  They really just took the samsung s8 and named it iphone x or whatever
Comment -  where is iPhone 9?
Comment -  Where's iPhone 9? Just skips to iPhone x. Lol okay nice math there Apple
Comment -  I want apple x
Comment -  Can you call me when you put some actual NEW tech in your next iPhone? Like the last iPhone that had some serious invocation was iPhone 6, with it 3D Touch. But since then all they done was, upgrade screen/processor/camera, all that stuff... yeah sure faceid bla bla bla, that's just a progr

In [76]:
print_bottom_10(hits_3)

Comment -  The strea was longer than my dick
Comment -  Cine-i moldovan  sau român pe aici?😂
Comment -  Se copiaron de motor o la
Comment -  Acil satılık böbrek
Comment -  Hallo wie geht's dir Tschüss bis nächstes Mal bye bye
Comment -  You will never find Losers who line up and pay good money for this crap line up to do charity or community work.
Comment -  birby
Comment -  Anda a la cancha bobo
Comment -  TREMENDO CELULARRR parfavar, ahre que no tengo dinero.
Comment -  fuck you beech


Still better results from sentence transformer pretrained model - all-MiniLM-L6-v2

Fine-tune sentence-transformers/all-MiniLM-L6-v2 model and see results.

In [77]:
miniLM = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')
pooling_lm = models.Pooling(miniLM.get_word_embedding_dimension(), 'cls')

model_lm = SentenceTransformer(modules=[miniLM, pooling])
model_lm

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
model_lm.fit(
    train_objectives=[(dataloader, loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model_lm.save('output/youtube-comments-tsdae-mini-lm')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
comments_embeddings_lm = get_embeddings(model_3, comments)
query_embedding_lm = get_embeddings(model_3, [query_text])